## 엑셀 내의 특정 시트를 CSV 파일로 변환

In [ ]:
import pandas as pd

def excel_sheet_to_csv(excel_file_path, sheet_name, output_csv_path):
    """
    엑셀 파일 내의 특정 시트를 CSV 파일로 변환하여 저장합니다.

    Args:
        excel_file_path (str): 원본 엑셀 파일의 경로 (예: '식약처_데이터.xlsx').
        sheet_name (str): CSV로 변환할 엑셀 시트의 이름.
        output_csv_path (str): 생성될 CSV 파일의 경로와 이름 (예: '영양성분_데이터.csv').
    """


    try:
        # 엑셀 파일에서 지정된 시트만 DataFrame으로 로드
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

        # DataFrame을 CSV 파일로 저장
        # index=False는 DataFrame의 인덱스를 CSV 파일에 저장하지 않도록 합니다.
        # encoding='utf-8-sig'는 한글 깨짐을 방지하는 BOM이 포함된 UTF-8 인코딩입니다.
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

        print(f"--- 시트 '{sheet_name}'이(가) '{output_csv_path}'로 성공적으로 변환되었습니다. ---")

    except FileNotFoundError:
        print(f"오류: 엑셀 파일 '{excel_file_path}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")
    except ValueError:
        print(f"오류: 엑셀 파일 내에 시트 '{sheet_name}'을(를) 찾을 수 없습니다. 시트 이름을 확인해주세요.")
    except Exception as e:
        print(f"CSV 변환 중 오류가 발생했습니다: {e}")

# --- 사용 예시 ---
if __name__ == "__main__":
    # 1. 원본 엑셀 파일 경로
    source_excel = '국가표준식품성분표.xlsx'

    # 2. CSV로 변환하고 싶은 시트 이름
    # 이전에 시트 목록을 확인했던 코드를 통해 정확한 시트 이름을 파악하세요.
    target_sheet = '국가표준식품성분 Database 10.0' # 예시 시트 이름입니다. 실제 파일의 시트 이름으로 변경해주세요!

    # 3. 새로 생성될 CSV 파일의 이름 (원하는 이름으로 지정)
    output_csv = '국가표준식품성분표.csv'

    excel_sheet_to_csv(source_excel, target_sheet, output_csv)

    

--- 시트 '국가표준식품성분 Database 10.0'이(가) '국가표준식품성분표.csv'로 성공적으로 변환되었습니다. ---


In [8]:
# 변환된 CSV 파일을 바로 확인하고 싶다면 아래 주석을 해제하세요.
print("\n--- 변환된 CSV 파일 상위 5개 행 확인 ---")
new_df = pd.read_csv(output_csv, encoding='utf-8-sig')
print(new_df.head())



--- 변환된 CSV 파일 상위 5개 행 확인 ---
   Unnamed: 0 *음영표시: 책자 수록 항목 Unnamed: 2 가식부 100g 당 (per 100g Edible Portion)  \
0  DB10.0\n색인   10개정 \n책자\n색인        식품군                                  식품명   
1         NaN             NaN        NaN                                  NaN   
2           1               1  곡류 및 그 제품                      귀리, 겉귀리, 도정, 생것   
3           2               2  곡류 및 그 제품                      귀리, 쌀귀리, 도정, 생것   
4           3             NaN  곡류 및 그 제품                  귀리, 쌀귀리, 대양, 도정, 생것   

  Unnamed: 4 일반성분 Proximates Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...  \
0         출처             에너지         수분        단백질        지방          회분  ...   
1        NaN            kcal          g          g          g          g  ...   
2   농진청('16)             380       11.5       11.4       7.35          2  ...   
3   농진청('20)             388       11.6      11.14        8.9        1.7  ...   
4   농진청('20)             386       11.8      11.51       8.78       1.81  ...

## 국가 표준 식품 성분표 전처리

In [ ]:
import pandas as pd
import numpy as np
import re # re 모듈 임포트했는지 다시 확인!

# CSV 파일 경로 (이전 단계에서 변환한 파일명으로 변경)
csv_file_path = '국가표준식품성분표.csv'

print("--- 1. 데이터 로드 및 초기 정제 시작 ---")
try:
    # header=1은 두 번째 행(0부터 시작하므로 인덱스 1)을 헤더로 사용
    # skipinitialspace=True는 콤마 뒤 공백을 무시
    df = pd.read_csv(csv_file_path, header=1, skipinitialspace=True, encoding='utf-8-sig')

    # 컬럼명 정제: 줄바꿈 제거, 양 끝 공백 제거, 'Unnamed' 컬럼 처리
    # 컬럼명을 문자열로 강제 변환 후 처리 (MultiIndex일 수 있으므로)
    cleaned_columns = []
    for col in df.columns:
        # MultiIndex인 경우 첫 번째 요소(상위 헤더) 사용, 아니면 그대로 사용
        col_name = str(col[0]) if isinstance(col, tuple) else str(col)
        cleaned_name = col_name.replace('\n', ' ').strip()
        cleaned_columns.append(cleaned_name)
    df.columns = cleaned_columns

    # 불필요한 'Unnamed' 컬럼 제거
    cols_to_drop = [col for col in df.columns if 'Unnamed' in col]
    # 'DB10.0' 등으로 시작하는 인덱스성 컬럼이 있다면 제거
    if df.columns[0].startswith('DB10') or df.columns[0].lower().startswith('색인'):
        cols_to_drop.append(df.columns[0])
    
    df = df.drop(columns=cols_to_drop, errors='ignore') # errors='ignore'로 없어도 오류 안 나게

    print("\n--- 초기 로드된 컬럼 목록 (정제 후) ---")
    print(df.columns.tolist()) # 리스트 형태로 출력하여 보기 쉽게

    # '식품명' 컬럼 존재 여부 확인 및 첫 유효한 데이터 행 찾기
    # 식약처 데이터에서 '식품명' 컬럼의 정확한 이름을 확인해야 합니다.
    # 예: '식품명', '식품이름', '식품군_식품명' 등
    food_name_col = None
    possible_food_name_cols = ['식품명', '식품이름', '식품군 식품명'] # 가능한 컬럼명 추가
    for col in possible_food_name_cols:
        if col in df.columns:
            food_name_col = col
            break
    
    if food_name_col is None:
        raise KeyError("핵심 컬럼 '식품명'을 찾을 수 없습니다. df.columns를 확인하여 정확한 컬럼명을 지정해주세요.")

    # '식품명' 컬럼이 NaN이 아닌 첫 행을 기준으로 데이터 자르기
    # 데이터 상단에 불필요한 설명 행들이 있을 수 있으므로 실제 데이터 시작점 찾기
    first_valid_row_idx = df[food_name_col].first_valid_index()
    if first_valid_row_idx is not None:
        df_processed = df.loc[first_valid_row_idx:].copy()
    else:
        # 만약 유효한 행을 찾지 못했다면, 데이터프레임이 비어있거나 '식품명' 컬럼이 모두 NaN일 수 있습니다.
        raise ValueError("데이터에 유효한 식품명 행이 없습니다. 파일 내용을 확인해주세요.")

    # 컬럼명 최종 변경 딕셔너리 (실제 데이터에 맞춰 정확히 수정 필요!)
    # 여기서는 식약처 통합 영양성분DB에서 일반적으로 쓰이는 컬럼명을 가정합니다.
    # df.columns.tolist()를 보고 매핑해야 합니다.
    rename_cols_map = {
        food_name_col: '식품명', # 찾은 식품명 컬럼을 '식품명'으로 통일
        '에너지': 'kcal',
        '단백질': 'protein_g',
        '지방': 'fat_g',
        '탄수화물': 'carb_g',
        '나트륨': 'sodium_mg',
        '식염상당량': 'salt_equivalent_g', # g 단위로 되어 있는지 확인 필요
        '수분': 'water_g', # 추가
        '회분': 'ash_g',   # 추가
        # 필요한 다른 영양성분 컬럼들도 여기에 추가하여 원하는 이름으로 변경
        # 예: '총 식이섬유': 'fiber_g', '총당류': 'sugar_g'
    }
    
    # df_processed에 실제로 존재하는 컬럼만 선택하여 rename_cols_map 업데이트
    actual_rename_cols_map = {old: new for old, new in rename_cols_map.items() if old in df_processed.columns}
    df_processed = df_processed.rename(columns=actual_rename_cols_map)

    # 영양 성분 컬럼들을 숫자로 변환 (이 부분이 특히 중요!)
    # '식품명' 컬럼은 제외하고, 변경된 컬럼명(values) 중에서 수치형인 것만 선택
    # df_processed에 존재하는 실제 컬럼들 중에서 수치형으로 변환할 대상만 명확히 지정합니다.
    
    # '식품명'을 제외한 모든 컬럼을 잠재적인 수치형 컬럼으로 간주하고 변환 시도
    # 실제 데이터의 컬럼 타입과 이름에 따라 이 리스트는 달라져야 합니다.
    potential_numeric_cols = [col for col in df_processed.columns if col not in ['식품명', '식품군']] # '식품군'도 문자열일 수 있으니 제외

    # 최종적으로 numeric_cols 리스트를 생성.
    # 이 리스트는 df_processed에 존재하며, '식품명'이 아닌 컬럼들 중에서 선택됩니다.
    numeric_cols = []
    for col in potential_numeric_cols:
        # 문자열에서 숫자와 점(.)만 남기고 모두 제거 후 숫자로 변환
        # pd.to_numeric이 변환할 수 없는 값은 NaN으로 만듭니다.
        df_processed[col] = pd.to_numeric(
            df_processed[col].astype(str).str.replace('[^0-9.]', '', regex=True),
            errors='coerce' # 변환 불가 시 NaN으로 처리
        )
        # NaN 값은 0으로 채웁니다. (데이터 특성에 따라 다른 전략 필요)
        df_processed[col] = df_processed[col].fillna(0)
        # 성공적으로 숫자로 변환된 컬럼만 numeric_cols에 추가
        # 모든 값이 0으로 채워졌더라도, 이전에 숫자로 변환 시도했으므로 추가
        if df_processed[col].dtype in ['float64', 'int64']:
             numeric_cols.append(col)

    print("\n--- 1. 초기 정제된 데이터 상위 5개 행 ---")
    print(df_processed.head())
    print("\n--- 1. 초기 정제된 데이터 컬럼 정보 ---")
    print(df_processed.info())

except KeyError as e:
    print(f"오류: 컬럼명을 찾을 수 없습니다. '{e}'. CSV 파일의 실제 컬럼명을 확인해주세요.")
    print("현재 데이터프레임의 컬럼 목록:")
    print(df.columns.tolist() if 'df' in locals() else "df가 로드되지 않았습니다.")
    raise # 오류를 다시 발생시켜 자세한 트레이스백 확인
except ValueError as e:
    print(f"데이터 정제 중 값 오류 발생: {e}")
    raise
except Exception as e:
    print(f"데이터 로드 및 초기 정제 중 예측하지 못한 오류 발생: {e}")
    raise

In [ ]:
# --- 2. 중복 식품명 처리 (핵심) ---
print("\n--- 2. 중복 식품명 처리 시작 ---")
try:
    # '식품명' 컬럼 정제: 상세 스펙 제거
    df_processed['식품명_정제'] = df_processed['식품명'].str.split(r'[,(]', expand=True)[0].str.strip()

    # nutrient_cols는 '1. 초기 로드 및 컬럼명 정제' 섹션에서 정의된 것을 사용
    # 만약 이 셀만 따로 실행한다면, df_processed에 맞는 numeric_cols를 수동 정의 필요
    # 예: numeric_cols = ['kcal', 'protein_g', 'fat_g', 'carb_g', 'sodium_mg', 'salt_equivalent_g', ...]
    if 'numeric_cols' not in locals(): # 이 부분이 없다면, 이전 셀이 실행되지 않은 것
        print("경고: 'numeric_cols'가 정의되지 않아, df_processed의 숫자형 컬럼을 가정합니다.")
        # df_processed.select_dtypes(include=np.number).columns.tolist() 로 실제 숫자 컬럼 찾기
        numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
        if '식품명' in numeric_cols: numeric_cols.remove('식품명') # 식품명은 숫자 아님
        if not numeric_cols:
            raise ValueError("수치형 영양소 컬럼을 찾을 수 없습니다. 'df_processed'의 컬럼을 확인하세요.")
        print(f"가정된 수치형 컬럼: {numeric_cols}")

    # 중복 처리 전략: '식품명_정제'가 같은 경우, 영양 성분은 평균값을 사용
    # agg_dict 생성: pd.NamedAgg를 사용하여 새 컬럼명 지정
    agg_dict = {}

    # 1. 수치형 컬럼은 평균 (새로운 컬럼명 = 기존 컬럼명)
    for col in numeric_cols:
        if col in df_processed.columns: # df_processed에 존재하는 컬럼만 추가
            agg_dict[col] = pd.NamedAgg(column=col, aggfunc='mean')

    # 2. 원본 '식품명'을 '식품명_원본'으로 보존 (첫 번째 값)
    if '식품명' in df_processed.columns:
        agg_dict['식품명_원본'] = pd.NamedAgg(column='식품명', aggfunc='first')

    # 3. '식품군' 컬럼도 보존 (만약 있다면)
    if '식품군' in df_processed.columns:
        agg_dict['식품군'] = pd.NamedAgg(column='식품군', aggfunc='first')

    print("Aggregation dictionary created:", agg_dict)
    # df_processed의 컬럼 목록을 다시 확인하여 agg_dict의 key들이 일치하는지 확인하는 것도 좋습니다.
    # print(df_processed.columns.tolist())


    # groupby와 agg 실행
    # '식품명_정제'는 groupby의 키가 되므로, agg_dict의 value로 포함되지 않습니다.
    # as_index=False를 사용하여 '식품명_정제'가 인덱스가 아닌 일반 컬럼으로 남도록
    df_deduplicated = df_processed.groupby('식품명_정제', as_index=False).agg(**agg_dict) # **agg_dict로 딕셔너리 언팩


    # 최종 컬럼명 변경 (기존 '식품명_정제'를 'item'으로)
    final_nutrition_df = df_deduplicated.rename(columns={'식품명_정제': 'item'})

    print("\n--- 2. 중복 처리된 데이터 상위 5개 행 ---")
    print(final_nutrition_df.head())
    print("\n--- 2. 중복 처리된 데이터 컬럼 정보 ---")
    print(final_nutrition_df.info())
    print(f"\n총 고유 식품명 개수: {len(final_nutrition_df)}")

except Exception as e:
    print(f"중복 식품명 처리 중 오류 발생: {e}")
    # 오류 발생 직전 df_processed의 상태 확인
    print("\n--- 오류 발생 직전 df_processed head ---")
    print(df_processed.head())
    print("\n--- 오류 발생 직전 df_processed info ---")
    print(df_processed.info())
    raise # 오류를 다시 발생시켜 자세한 트레이스백 확인

### CSV 파일로 저장

In [ ]:
# 1. 정제된 데이터를 CSV 파일로 저장
output_csv_file_path = '전처리_국가표준식품성분표.csv' # 저장할 파일명 지정
final_nutrition_df.to_csv(output_csv_file_path, index=False, encoding='utf-8-sig')

print(f"\n정제된 영양 성분 데이터가 '{output_csv_file_path}' 파일로 성공적으로 저장되었습니다.")

In [ ]:
import pandas as pd
# 2. (선택 사항) 저장된 CSV 파일에서 다시 불러와 확인
# 나중에 작업을 시작할 때는 이 CSV 파일을 바로 로드하여 사용할 수 있습니다.
df_from_csv = pd.read_csv('전처리_국가표준식품성분표.csv', encoding='utf-8-sig')
print("\n--- CSV 파일에서 다시 불러온 데이터 상위 5개 행 ---")
print(df_from_csv.head())
print("\n--- CSV 파일에서 다시 불러온 데이터 컬럼 정보 ---")
print(df_from_csv.info())